In [3]:
import pandas as pd 
from datetime import datetime
from datetime import date
from collections import Counter
import requests
import json

In [4]:
#API 
# https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta -----> base url

headers = {'Content-Type': 'application/json', 'Access-Control-Request-Headers': '*','api-key': 'quuKmslQouhgHNNdtya30WaRxNhXxVcvD5WJlJ0vGmsa9Z9ZccSV4eKast0OjAHb'}
Payload_template = json.dumps({"collection": "bankStatement", "database": "Modev","dataSource": "BiCluster", })


insertOne_url = "https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta/action/insertOne"
Request_Body_insert_one = {"document": '{ ... }'}

insertMany_url = "https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta/action/insertMany"
Request_Body_insert_many = {"documents": ['{ ... }', '{...}']}

findOne_url = "https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta/action/findOne"
Request_Body_find_one = {"filter": '<query filter>',"projection": '<projection>'}

findAll_url = "https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta/action/find"
Request_Body_find_many = {"filter": '<query filter>',"projection": '<projection>',"sort": '<sort expression>', "limit": '<number>', "skip": '<number>'}

updateOne_url = "https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta/action/updateOne"
Request_Body_update_one = {"filter": '<query filter>', "update": '{ ... }', "upsert": 'true|false'}

updateMany_url = "https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta/action/updateMany"
Request_Body_update_many = {"filter": '<query filter>', "update": '{ ... }', "upsert": 'true|false'}

replaceOne_url = "https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta/action/replaceOne"
Request_Body_replace_one = {"filter": '<query filter>', "replacement": '{ ... }', "upsert": 'true|false'}

deleteOne_url = "https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta/action/deleteOne"
Request_Body_delete_one = {"filter": '<query filter>'}

deleteMany_url = "https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta/action/deleteMany"
Request_Body_delete_many = {"filter": '<query filter>'}



# response = requests.request("POST", (base_url + action), headers=headers, data=(Payload_template + Request_body))
# print(response.text)


In [5]:
bank_path = './bank.csv'
credit_card = './card.csv'
bill = './bill.csv'
bills = './bills.csv'
upWork = './upWork.csv'
QuickBooks_Bank = './QuickBooks_Bank.csv'
QuickBooks_Cat = './QuickBooks_Bank_catagorized.csv'
budget = './budget.xlsx'

opening_balance = 148136.99

start_date = datetime(2022,2,1,1,0,0)
dt = start_date.strftime("%m/%d/%Y")

### Bank & Budget

In [20]:
budget_df = pd.read_csv(bank_path)
budget_df['Date'] = pd.to_datetime(budget_df['Date'])

budget_df['Amount'] = budget_df['Amount'].map(lambda Amount: (Amount.replace("$", "")).replace("+", ""))
budget_df['Amount'] = budget_df['Amount'].map(lambda Amount: (Amount.replace("(", "-")).replace(")", ""))
budget_df['Amount'] = budget_df['Amount'].map(lambda Amount: float(Amount))

budget_df.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  98 non-null     datetime64[ns]
 1   Transaction Type      98 non-null     object        
 2   Check Number          54 non-null     float64       
 3   Description           98 non-null     object        
 4   Amount                98 non-null     float64       
 5   Daily Posted Balance  71 non-null     object        
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 4.7+ KB


In [34]:
# budget_df
# bank_data = Counter()

date = [_ for _ in budget_df['Date']]
description = [_ for _ in budget_df['Description']]
amount = [_ for _ in budget_df['Amount']]

for i in range(90, len(date)):
    doc = {"Date": str(date[i]),"Description": description[i], "Amount": amount[i]}
    payload = json.dumps({"collection": "bankStatement", "budget": "Modev","dataSource": "BiCluster",
                          "document": doc})
    response = requests.request("POST", insertOne, headers=headers, data=payload)
    print(i, response.text)
# bank_data

{"insertedId":"62430501ae4dee0fb1cd7c08"}
{"insertedId":"62430502ae4dee0fb1cd7cd6"}
{"insertedId":"62430503648201e45c4f4096"}
{"insertedId":"62430504648201e45c4f4182"}
{"insertedId":"6243050596f2619fd6fe258f"}
{"insertedId":"624305d696f2619fd6ff5922"}
{"insertedId":"624305d75b345cd5224c219f"}
{"insertedId":"624305d92acaaad67b7e9e68"}


In [7]:
budget_ = pd.read_excel(budget, sheet_name="Modev Budget")
# print(budget_df.index)
class_p = [_ for _ in budget_['Project Class']]
acc = [_ for _ in budget_['Description']]
acc_type = [_ for _ in budget_['Account Type']]
q1 = [_ for _ in budget_['Q1']]
q2 = [_ for _ in budget_['Q2']]
q3 = [_ for _ in budget_['Q3']]
q4 = [_ for _ in budget_['Q4']]
total = [_ for _ in budget_['Total Budget']]

for i in range(len(class_p)):
    doc = {"Class": class_p[i],"Account": acc[i], "Account_Type": acc_type[i], "Q1": q1[i], 
           "Q2": q2[i], "Q3": q3[i], "Q4": q4[i], "Total": total[i]}
    payload = json.dumps({"collection": "budget", "database": "Modev","dataSource": "BiCluster",
                          "document": doc})
    response = requests.request("POST", insertOne, headers=headers, data=payload)
    print(i, response.text)


0 {"insertedId":"6243f5bfc344fc640c9252af"}
1 {"insertedId":"6243f5c098a45a4e41ca8654"}
2 {"insertedId":"6243f5c3c344fc640c925591"}
3 {"insertedId":"6243f5c498a45a4e41ca8a06"}
4 {"insertedId":"6243f5c9a84e73d230c43af8"}
5 {"insertedId":"6243f5cac344fc640c925b78"}
6 {"insertedId":"6243f5cba84e73d230c43c1a"}
7 {"insertedId":"6243f5cca84e73d230c43d18"}


In [17]:
card = pd.read_csv(credit_card)
card['Date'] = pd.to_datetime(card['Date'])
date = [_ for _ in card['Date']]
acc = [_ for _ in card['Account #']]
description = [_ for _ in card['Description']]
amount = [_ for _ in card['Amount']]

docs = []
for i in range(len(date)):
    doc = {"Date": str(date[i]),"Account": acc[i], "Description": description[i], "Amount": amount[i]}
    docs.append(doc)

payload = json.dumps({"collection": "credit", "database": "Modev","dataSource": "BiCluster",
                          "documents": docs})
response = requests.request("POST", insertMany, headers=headers, data=payload)
print(response.text)

{"insertedIds":["6243f954a84e73d230c76f1c","6243f954a84e73d230c76f1d","6243f954a84e73d230c76f1e","6243f954a84e73d230c76f1f","6243f954a84e73d230c76f20","6243f954a84e73d230c76f21","6243f954a84e73d230c76f22","6243f954a84e73d230c76f23","6243f954a84e73d230c76f24","6243f954a84e73d230c76f25","6243f954a84e73d230c76f26","6243f954a84e73d230c76f27","6243f954a84e73d230c76f28","6243f954a84e73d230c76f29","6243f954a84e73d230c76f2a","6243f954a84e73d230c76f2b","6243f954a84e73d230c76f2c","6243f954a84e73d230c76f2d","6243f954a84e73d230c76f2e","6243f954a84e73d230c76f2f","6243f954a84e73d230c76f30","6243f954a84e73d230c76f31","6243f954a84e73d230c76f32","6243f954a84e73d230c76f33","6243f954a84e73d230c76f34","6243f954a84e73d230c76f35","6243f954a84e73d230c76f36","6243f954a84e73d230c76f37","6243f954a84e73d230c76f38","6243f954a84e73d230c76f39","6243f954a84e73d230c76f3a","6243f954a84e73d230c76f3b","6243f954a84e73d230c76f3c","6243f954a84e73d230c76f3d","6243f954a84e73d230c76f3e","6243f954a84e73d230c76f3f","6243f954a84

In [14]:
revenue = budget_df[budget_df['Account Type'] == "Revenue"]
expense = budget_df[budget_df['Account Type'] == "Expense"]

In [19]:
revenue = budget_df[budget_df['Account Type'] == "Revenue"]
expense = budget_df[budget_df['Account Type'] == "Expense"]
diff = revenue["Total Budget"].sum() - expense["Total Budget"].sum()
diff

406373

In [11]:
expense

,Ref,Project Class,Description,Account Type,Q1,Q2,Q3,Q4,Total Budget
3,Exp_001,VOICE SUMMIT,COGS,Expense,0,0,0,0,750000
4,Exp_002,W3 CONFERENCE,COGS,Expense,0,0,0,0,500000
5,Exp_003,G&A,LABOR,Expense,157651,157651,157651,157651,630604
6,Exp_004,G&A,MARKETING,Expense,0,0,0,0,13500
7,Exp_005,G&A,ADMIN/GENERAL,Expense,0,0,0,0,199523


In [101]:
bank = pd.read_csv(bank_path)
bank['Amount'] = bank['Amount'].map(lambda Amount: Amount.replace("+", ""))
bank['Amount'] = bank['Amount'].map(lambda Amount: Amount.replace("$", ""))
bank['Amount'] = bank['Amount'].map(lambda Amount: Amount.replace("(", "-"))
bank['Amount'] = bank['Amount'].map(lambda Amount: Amount.replace(")", ""))
bank['Amount'] = bank['Amount'].map(lambda Amount: float(Amount))
bank['Date'] = pd.to_datetime(bank['Date'])

# income = bank[bank['Amount']>0]
# income = "{:,.2f}".format(bank[bank['Amount']>0].Amount.sum())
# print(income)
# withdrawals = "{:,.2f}".format(bank[bank['Amount']<0].Amount.sum())
# print(withdrawals)
# bal_calc = opening_balance + bank[bank['Amount']>0].Amount.sum() + bank[bank['Amount']<0].Amount.sum()
# balance = "{:,.2f}".format(bal_calc)
# print(balance)
# bank.to_csv("bank_clean.csv")
bank = bank[bank["Date"]>start_date]
bank

,Date,Transaction Type,Check Number,Description,Amount,Daily Posted Balance
35,2022-02-02,Debit,0.0,GLOBAL PAYMENTS GLOBAL STL 8788210011971 ...,-4.95,$42589.46
36,2022-02-02,Debit,0.0,AUTHNET GATEWAY BILLING 121118231 ...,-25.00,$42564.46
37,2022-02-02,Debit,0.0,GUSTO FEE 385298 6SEMJR4I314 ...,-51.00,$42513.46
38,2022-02-02,Debit,0.0,BILL.COM PAYABLES 016JMLVWS23RP9I ...,-3750.00,$38763.46
39,2022-02-04,DEPOSIT,0.0,DEPOSIT ...,10343.61,$49107.07
...,...,...,...,...,...,...
93,2022-03-16,Debit,NaN,Industriou Industrious tech ST-M ...,-50.00,$36312.49
94,2022-03-17,Debit,NaN,LIFE USAA.COM PAY EXT D033345925 ...,-290.38,$36022.11
95,2022-03-18,Credit,NaN,2367 WillowTree MoDevNetwork LLC ...,40000.00,NaN
96,2022-03-18,Debit,NaN,DEBIT PayPal ModevNetwork LLC AC ...,-2253.17,$73768.94


In [77]:
Bills = pd.read_csv(bills)
Bills


,Invoice #,Vendor,Chart of account,Class,Created Date,Invoice Date,Due Date,Invoice Amount,Payment Status,Approval Status
0,102721,John Uribe,Subcontracted Services,VOICE Talks NA 2021,01/24/22,11/14/21,12/14/21,"1,500.00",Paid,Approved
1,510631-M002,Adam Rarey,Cost of labor - COS,VOICE Talks NA 2021,01/06/22,01/05/22,01/20/22,"2,500.00",Paid,Approved
2,01/17/2022,Adam Rarey,Cost of labor - COS,VOICE Talks NA 2021,01/24/22,01/17/22,02/01/22,"2,500.00",Unpaid,Denied
3,1559763540133,Ryan O' Leary,Subcontracted Services,VOICE Talks NA 2021,01/24/22,01/19/22,02/03/22,158.00,Paid,Approved
4,01/22/2022,Adam Rarey,Cost of labor - COS,VOICE Talks NA 2021,01/24/22,01/22/22,02/06/22,"3,750.00",Paid,Approved
5,12/10/2021,Bridget Collins,Subcontracted Services,VOICE Summit 2022,02/02/22,12/10/21,12/25/21,"1,500.00",Paid,Approved
6,64,Dan Whiting,Subcontracted Services,VOICE Summit 2022,02/24/22,12/30/21,12/30/21,850.00,Paid,Approved
7,64,Dan Whiting,Subcontracted Services,VOICE Summit 2022,02/02/22,12/30/21,01/14/22,850.00,Paid,Approved
8,02/24/2022,Bridget Collins,Subcontracted Services,VOICE Summit 2022,02/24/22,02/24/22,02/24/22,"1,500.00",Paid,Approved
9,VOICE22-02,Howard Associates,Subcontracted Services,VOICE Summit 2022,02/22/22,02/10/22,03/12/22,"4,500.00",Paid,Approved


In [83]:
bill_df = pd.read_csv(bill)
bill_df['PAYMENT AMOUNT'] = bill_df['PAYMENT AMOUNT'].map(lambda PAYMENT: float(PAYMENT.replace(",", "")))
date_string = '2022-01-01'
start = datetime.strptime(date_string, '%Y-%m-%d')
end = datetime.today()
bill_df['PROCESS DATE'] = pd.to_datetime(bill_df['PROCESS DATE'])
bill_df = bill_df[(bill_df['PROCESS DATE'] >= start) & (bill_df['PROCESS DATE'] <=end )]
print(bill_df.shape)
bill_df['PAYMENT AMOUNT'].sum()

(37, 8)


126391.08

In [61]:
bank_description_01  = 'Bill.com'
bank_description_02 = 'WIRE REF#'
bank_description_02_01 = 'BUSONLINE DOMESTIC WIRE'

bank_wire = bank.loc[lambda d: d['Description'].str.lower().str.contains('wire ref')]
descriptions = [ _ for _ in bank.loc[lambda d: d['Description'].str.lower().str.contains('wire ref')]['Description']]
descriptions

# print(len(descriptions))
def has_bill_com(text):
    return "WIRE REF#" in text

bill_com = [description for description in descriptions if has_bill_com]
# bill_com
# len(bill_com)
# [next(bill_com) for i in range(1)]
descriptions
bank_wire = bank.loc[lambda d: d['Description'].str.lower().str.contains('wire ref')]
bank_bill_com = bank.loc[lambda d: d['Description'].str.lower().str.contains('bill.com')]

bank_bill = pd.concat([bank_bill_com, bank_wire])
bank_bill = bank_bill[bank_bill['Amount']<0]
print(bank_bill["Amount"].sum())


-119079.07999999999


In [64]:
diff = bill_df['PAYMENT AMOUNT'].sum() + bank_bill["Amount"].sum()
diff

7312.000000000015

In [70]:
QuickBooksBank = pd.read_csv(QuickBooks_Bank)
QuickBooksBank['Category or Match'].unique()

array(['American Express', 'Payroll Expenses:Employer Payroll Taxes',
       'Interest Expense', 'Merchant Account Fees',
       'Uncategorized Expense'], dtype=object)

In [72]:
QuickBooksBank

,DATE,DESCRIPTION,Payee,Category or Match,Class,SPENT,RECEIVED
0,03/22/2022,American Express,NaN,American Express,NaN,"$21,127.79",NaN
1,03/15/2022,"Modevnetwork, LLC",Gusto,Payroll Expenses:Employer Payroll Taxes,G&A,"$10,340.00",NaN
2,03/03/2022,"Modevnetwork, LLC",Gusto,Payroll Expenses:Employer Payroll Taxes,G&A,"$10,000.00",NaN
3,02/28/2022,Comm Loans Truist,NaN,Interest Expense,NaN,"$4,326.90",NaN
4,03/02/2022,WIRE REF# 20220302-00004960 BUSONLINE DOMESTIC...,NaN,Merchant Account Fees,NaN,"$4,170.00",NaN
5,03/15/2022,Wire Ref Busonline,NaN,Merchant Account Fees,NaN,"$4,170.00",NaN
6,03/11/2022,"Modevnetwork, LLC",Gusto,Payroll Expenses:Employer Payroll Taxes,G&A,"$4,100.00",NaN
7,02/28/2022,"Modevnetwork, LLC",Gusto,Payroll Expenses:Employer Payroll Taxes,G&A,"$3,514.06",NaN
8,03/03/2022,WIRE REF# 20220303-00013969 BUSONLINE DOMESTIC...,NaN,Merchant Account Fees,NaN,"$3,000.00",NaN
9,02/25/2022,Comm Loans Truist,NaN,Interest Expense,NaN,"$1,686.71",NaN


In [69]:
QuickBooksBank[QuickBooksBank['Category or Match']=='Merchant Account Fees']

,DATE,DESCRIPTION,Payee,Category or Match,Class,SPENT,RECEIVED
4,03/02/2022,WIRE REF# 20220302-00004960 BUSONLINE DOMESTIC...,NaN,Merchant Account Fees,NaN,"$4,170.00",NaN
5,03/15/2022,Wire Ref Busonline,NaN,Merchant Account Fees,NaN,"$4,170.00",NaN
8,03/03/2022,WIRE REF# 20220303-00013969 BUSONLINE DOMESTIC...,NaN,Merchant Account Fees,NaN,"$3,000.00",NaN
12,03/03/2022,WIRE REF# 20220303-00014335 BUSONLINE DOMESTIC...,NaN,Merchant Account Fees,NaN,$850.00,NaN


In [71]:
QuickBooksBank[QuickBooksBank['Category or Match']=='Uncategorized Expense']


,DATE,DESCRIPTION,Payee,Category or Match,Class,SPENT,RECEIVED
14,03/11/2022,Crcardpmt Capital One,CAPITAL ONE FELLOWS,Uncategorized Expense,NaN,$468.00,NaN
15,03/11/2022,Crcardpmt Capital One,CAPITAL ONE FELLOWS,Uncategorized Expense,NaN,$133.00,NaN


### UPwork

In [7]:
upWork_df = pd.read_csv(upWork)
upWork_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Date                  260 non-null    object 
 1   Activity ID           256 non-null    object 
 2   Contract              260 non-null    object 
 3   Talent                260 non-null    object 
 4   Hours                 256 non-null    float64
 5   Amount                260 non-null    float64
 6   Payment Type          4 non-null      object 
 7   Activity Description  256 non-null    object 
dtypes: float64(2), object(6)
memory usage: 16.4+ KB


In [8]:
upWork_df.head(10)

,Date,Activity ID,Contract,Talent,Hours,Amount,Payment Type,Activity Description
0,2022-01-03,GeneralModevWork,Quickbooks Bookkeeping Expert (27865710),Vema Reddy (reddy-vema),0.500000,5.000000,NaN,G&A Work Not Specific to one single project
1,2022-01-03,VOICE_Talks_North_America,Hubspot Email Marketing Manager (26780046),Sheena Graham (grahamsheena),5.000000,275.000000,NaN,VOICE Talks North America
2,2022-01-03,VOICE_Talks_North_America,Writer / Editor for leading AI tech event seri...,Marc Dahan (pixelnoise),6.000000,270.000000,NaN,VOICE Talks North America
3,2022-01-03,VOICESUMMIT2021,Tech Event Marketing Manager (28449307),Rachel Keller (rachel-keller),1.333333,66.666667,NaN,VOICE Summit 2021
4,2022-01-04,VOICE_Talks_North_America,Hubspot Email Marketing Manager (26780046),Sheena Graham (grahamsheena),5.000000,275.000000,NaN,VOICE Talks North America
5,2022-01-05,NaN,Seeking Social Media Rock Star for VOICE Talks...,Stephanie Hidalgo (stephhidalgo),NaN,1000.000000,Fixed Price Payment,NaN
6,2022-01-05,GeneralModevWork,UI Designer (Hubspot experience preferred) (28...,Adam Mitchell (davidmitcha),0.500000,25.000000,NaN,G&A Work Not Specific to one single project
7,2022-01-05,VOICE_Talks_North_America,Hubspot Email Marketing Manager (26780046),Sheena Graham (grahamsheena),5.000000,275.000000,NaN,VOICE Talks North America
8,2022-01-06,VOICE_Talks_North_America,Hubspot Email Marketing Manager (26780046),Sheena Graham (grahamsheena),3.000000,165.000000,NaN,VOICE Talks North America
9,2022-01-06,VOICESUMMIT2021,Hubspot Developer - Immediate opening (24307811),Rob Marriott (902websolutions),1.833333,91.666667,NaN,VOICE Summit 2021


In [9]:
upWork_df['Date'] = pd.to_datetime(upWork_df['Date'])
upWork_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  260 non-null    datetime64[ns]
 1   Activity ID           256 non-null    object        
 2   Contract              260 non-null    object        
 3   Talent                260 non-null    object        
 4   Hours                 256 non-null    float64       
 5   Amount                260 non-null    float64       
 6   Payment Type          4 non-null      object        
 7   Activity Description  256 non-null    object        
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 16.4+ KB


In [20]:
date = [_ for _ in upWork_df['Date']]
acc = [str(_) for _ in upWork_df['Activity ID']]
description = [str(_) for _ in upWork_df['Activity Description']]
contract = [_ for _ in upWork_df['Contract']]
amount = [_ for _ in upWork_df['Amount']]

docs = []
for i in range(len(date)):
    doc = {"Date": str(date[i]),"Class": acc[i], "Description": description[i], 'Contract':contract[i], "Amount": amount[i]}
    docs.append(doc)
print(len(docs))

payload = json.dumps({"collection": "upwork", "database": "Modev","dataSource": "BiCluster", "documents": docs})

response = requests.request("POST", insertMany_url, headers=headers, data=payload)
print(response)

260
<Response [201]>


### Bill.com bills

In [26]:
bill_payments = pd.read_csv(bill)
bill_payments.info(verbose=True)
bill_payments.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   CONFIRMATION #  43 non-null     object
 1   VENDOR          43 non-null     object
 2   INVOICE #       43 non-null     object
 3   CURRENCY        43 non-null     object
 4   PAYMENT AMOUNT  43 non-null     object
 5   PROCESS DATE    43 non-null     object
 6   PAYMENT TYPE    43 non-null     object
 7   PAYMENT STATUS  43 non-null     object
dtypes: object(8)
memory usage: 2.8+ KB


,CONFIRMATION #,VENDOR,INVOICE #,CURRENCY,PAYMENT AMOUNT,PROCESS DATE,PAYMENT TYPE,PAYMENT STATUS
0,P22033001 - 9273241,Howard Associates,VOICE22-01,USD,"4,500.00",03/31/22,ePayment,Processed
1,P22032801 - 8604367,Madisyn Bozarth,019,USD,"3,000.00",03/31/22,ePayment,Processed
2,P22032801 - 8604369,MFK Communications LLC,1076,USD,400.00,03/31/22,ePayment,Processed


In [27]:
bills_df = pd.read_csv(bills)
bills_df.info(verbose=True)
bills_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Invoice #         82 non-null     object
 1   Vendor            82 non-null     object
 2   Chart of account  82 non-null     object
 3   Class             82 non-null     object
 4   Created Date      82 non-null     object
 5   Invoice Date      82 non-null     object
 6   Due Date          82 non-null     object
 7   Invoice Amount    82 non-null     object
 8   Payment Status    82 non-null     object
 9   Approval Status   82 non-null     object
dtypes: object(10)
memory usage: 6.5+ KB


,Invoice #,Vendor,Chart of account,Class,Created Date,Invoice Date,Due Date,Invoice Amount,Payment Status,Approval Status
0,1559763540127,Ryan O' Leary,Subcontracted Services,VOICE Talks NA 2021,12/02/21,11/23/21,12/08/21,"1,500.00",Paid,Approved
1,102721,John Uribe,Subcontracted Services,VOICE Talks NA 2021,01/24/22,11/14/21,12/14/21,"1,500.00",Paid,Approved
2,1559763540129,Ryan O' Leary,Subcontracted Services,VOICE Talks NA 2021,12/02/21,11/30/21,12/15/21,184.99,Paid,Approved


In [29]:
paid_invoices =[_ for _ in  bill_payments['INVOICE #']]
invoices = [_ for _ in bills_df['Invoice #']]

processing_date = []

for inv in invoices:
    if inv in paid_invoices:
        date = bill_payments[bill_payments['INVOICE #'] == inv]['PROCESS DATE'].values[0]
        processing_date.append(date)
    else:
        date = '12/31/21'	
        processing_date.append(date)

bills_df["Processing Date"] = processing_date
bills_df.head(3)


,Invoice #,Vendor,Chart of account,Class,Created Date,Invoice Date,Due Date,Invoice Amount,Payment Status,Approval Status,Processing Date
0,1559763540127,Ryan O' Leary,Subcontracted Services,VOICE Talks NA 2021,12/02/21,11/23/21,12/08/21,"1,500.00",Paid,Approved,12/31/21
1,102721,John Uribe,Subcontracted Services,VOICE Talks NA 2021,01/24/22,11/14/21,12/14/21,"1,500.00",Paid,Approved,01/25/22
2,1559763540129,Ryan O' Leary,Subcontracted Services,VOICE Talks NA 2021,12/02/21,11/30/21,12/15/21,184.99,Paid,Approved,12/31/21


In [31]:
bills_df['Processing Date'] = pd.to_datetime(bills_df['Processing Date'])
bills_df['Created Date'] = pd.to_datetime(bills_df['Created Date'])
bills_df['Invoice Date'] = pd.to_datetime(bills_df['Invoice Date'])
bills_df['Due Date'] = pd.to_datetime(bills_df['Due Date'])
bills_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Invoice #         82 non-null     object        
 1   Vendor            82 non-null     object        
 2   Chart of account  82 non-null     object        
 3   Class             82 non-null     object        
 4   Created Date      82 non-null     datetime64[ns]
 5   Invoice Date      82 non-null     datetime64[ns]
 6   Due Date          82 non-null     datetime64[ns]
 7   Invoice Amount    82 non-null     object        
 8   Payment Status    82 non-null     object        
 9   Approval Status   82 non-null     object        
 10  Processing Date   82 non-null     datetime64[ns]
dtypes: datetime64[ns](4), object(7)
memory usage: 7.2+ KB


In [37]:
bills_df['Invoice Amount '] = bills_df['Invoice Amount '].map(lambda Amount: float(Amount.replace(",", "")))
bills_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82 entries, 0 to 81
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Invoice #         82 non-null     object        
 1   Vendor            82 non-null     object        
 2   Chart of account  82 non-null     object        
 3   Class             82 non-null     object        
 4   Created Date      82 non-null     datetime64[ns]
 5   Invoice Date      82 non-null     datetime64[ns]
 6   Due Date          82 non-null     datetime64[ns]
 7   Invoice Amount    82 non-null     float64       
 8   Payment Status    82 non-null     object        
 9   Approval Status   82 non-null     object        
 10  Processing Date   82 non-null     datetime64[ns]
dtypes: datetime64[ns](4), float64(1), object(6)
memory usage: 7.2+ KB


In [40]:
invoice = [_ for _ in bills_df['Invoice #']]
vendor = [_ for _ in bills_df['Vendor']]
acc = [_ for _ in bills_df['Chart of account']]
prj = [_ for _ in bills_df['Class']]
crt_dt = [str(_) for _ in bills_df['Created Date']]
inv_dt = [str(_) for _ in bills_df['Invoice Date']]
due_dt = [str(_) for _ in bills_df['Due Date']]
amount = [_ for _ in bills_df['Invoice Amount ']]
stat = [_ for _ in bills_df['Payment Status']]
apr = [_ for _ in bills_df['Approval Status']]
prc_dt = [str(_) for _ in bills_df['Processing Date']]

docs = []
for i in range(8, len(invoice)):
    doc = {"Date": str(prc_dt[i]), "Class": prj[i], "Account": acc[i], 'Vendor':vendor[i], "Invoice":invoice[i], 'Invoice Date':inv_dt[i], 'Due Date':due_dt[i], 'Created Date':crt_dt[i], "Amount": amount[i], "Approval": apr[i], 'Status': stat[i]}
    docs.append(doc)
print(len(docs))

payload = json.dumps({"collection": "bills", "database": "Modev","dataSource": "BiCluster", "documents": docs})

response = requests.request("POST", insertMany_url, headers=headers, data=payload)
print(response)

74
<Response [201]>


### Connection testing 

In [42]:
findAll_url = "https://data.mongodb-api.com/app/data-wqlxh/endpoint/data/beta/action/find"
Payload = json.dumps({"collection": "bills", "database": "Modev","dataSource": "BiCluster", "filter": {}})
response = requests.request("POST", findAll_url, headers=headers, data=Payload)
if response.ok:
    response_json = response.json()
    colomun = [_ for _ in response_json['documents'][0].keys()]
    df_data = {}
    for col in colomun:
        df_data[col] = []
        
    for doc in response_json['documents']:
        for col in colomun:
            df_data[col].append(doc[col])
    df = pd.DataFrame(df_data)
    return df
else:
    return None


<function dict.keys>

In [44]:
colomun = [_ for _ in response_json['documents'][0].keys()]
colomun

['_id',
 'Date',
 'Class',
 'Account',
 'Vendor',
 'Invoice',
 'Invoice Date',
 'Due Date',
 'Created Date',
 'Amount',
 'Approval',
 'Status']

In [55]:
df_data = {}
for col in colomun:
    df_data[col] = []
    
for doc in response_json['documents']:
    for col in colomun:
        df_data[col].append(doc[col])
df = pd.DataFrame(df_data)

In [57]:

df.head(10)

,_id,Date,Class,Account,Vendor,Invoice,Invoice Date,Due Date,Created Date,Amount,Approval,Status
0,62455393e9d255ab42f82ddc,2021-12-31 00:00:00,VOICE Talks NA 2021,Subcontracted Services,Ryan O' Leary,1559763540127,2021-11-23 00:00:00,2021-12-08 00:00:00,2021-12-02 00:00:00,1500.00,Approved,Paid
1,62455393e9d255ab42f82ddd,2022-01-25 00:00:00,VOICE Talks NA 2021,Subcontracted Services,John Uribe,102721,2021-11-14 00:00:00,2021-12-14 00:00:00,2022-01-24 00:00:00,1500.00,Approved,Paid
2,62455393e9d255ab42f82dde,2021-12-31 00:00:00,VOICE Talks NA 2021,Subcontracted Services,Ryan O' Leary,1559763540129,2021-11-30 00:00:00,2021-12-15 00:00:00,2021-12-02 00:00:00,184.99,Approved,Paid
3,62455393e9d255ab42f82ddf,2022-01-25 00:00:00,VOICE Talks NA 2021,Cost of labor - COS,Adam Rarey,12/06/2021,2021-12-06 00:00:00,2021-12-21 00:00:00,2021-12-09 00:00:00,2500.00,Approved,Paid
4,62455393e9d255ab42f82de0,2021-12-31 00:00:00,VOICE Talks NA 2021,Cost of labor - COS,Ryan O' Leary,1559763540131,2021-12-08 00:00:00,2021-12-23 00:00:00,2021-12-09 00:00:00,3000.00,Approved,Paid
5,62455393e9d255ab42f82de1,2021-12-31 00:00:00,VOICE Talks NA 2021,Subcontracted Services,Ryan O' Leary,1559763540132,2021-12-08 00:00:00,2021-12-23 00:00:00,2021-12-09 00:00:00,173.59,Approved,Paid
6,62455393e9d255ab42f82de2,2021-12-31 00:00:00,VOICE Talks NA 2021,Subcontracted Services,Leyla Productions,3313,2021-12-09 00:00:00,2021-12-24 00:00:00,2021-12-11 00:00:00,2700.00,Approved,Paid
7,62455393e9d255ab42f82de3,2021-12-31 00:00:00,VOICE Talks NA 2021,Cost of labor - COS,Adam Rarey,12/15/2021,2021-12-15 00:00:00,2021-12-30 00:00:00,2021-12-18 00:00:00,2500.00,Approved,Paid
8,624553d5ab4a51c8cceacd35,2022-01-14 00:00:00,VOICE Talks NA 2021,Cost of labor - COS,Adam Rarey,510631-M002,2022-01-05 00:00:00,2022-01-20 00:00:00,2022-01-06 00:00:00,2500.00,Approved,Paid
9,624553d5ab4a51c8cceacd36,2021-12-31 00:00:00,VOICE Talks NA 2021,Cost of labor - COS,Adam Rarey,01/17/2022,2022-01-17 00:00:00,2022-02-01 00:00:00,2022-01-24 00:00:00,2500.00,Denied,Unpaid


In [53]:
if response.ok:
    colomun = [_ for _ in response_json['documents'][0].keys()]
    print(colomun)

['_id', 'Date', 'Class', 'Account', 'Vendor', 'Invoice', 'Invoice Date', 'Due Date', 'Created Date', 'Amount', 'Approval', 'Status']
